[View in Colaboratory](https://colab.research.google.com/github/Curiousss/FLD/blob/master/FLDNetwork.ipynb)

In [1]:
!wget https://ibug.doc.ic.ac.uk/download/annotations/300w_cropped.zip

--2018-09-02 05:49:10--  https://ibug.doc.ic.ac.uk/download/annotations/300w_cropped.zip
Resolving ibug.doc.ic.ac.uk (ibug.doc.ic.ac.uk)... 146.169.13.66
Connecting to ibug.doc.ic.ac.uk (ibug.doc.ic.ac.uk)|146.169.13.66|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: http://ibug.doc.ic.ac.uk/download/annotations/300w_cropped.zip/ [following]
URL transformed to HTTPS due to an HSTS policy
--2018-09-02 05:49:11--  https://ibug.doc.ic.ac.uk/download/annotations/300w_cropped.zip/
Reusing existing connection to ibug.doc.ic.ac.uk:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘300w_cropped.zip’

300w_cropped.zip        [ <=>                ]   2.82K  --.-KB/s    in 0s      

2018-09-02 05:49:12 (133 MB/s) - ‘300w_cropped.zip’ saved [2885]



In [3]:
!pip install dropbox

    100% |████████████████████████████████| 481kB 6.0MB/s 


In [0]:
import dropbox
access_token = 'DFUYAK9nil8AAAAAAAABp2DVoVsW3ha1K5iQiYWMtjh2QMjW8v8x1BjnpgToJb7B' # https://www.dropbox.com/developers/apps
dbx = dropbox.Dropbox(access_token)


In [0]:
from dropbox.files import WriteMode

md, response = dbx.files_download('/300w_cropped.zip')


In [13]:
!wget https://www.dropbox.com/s/9j2om0gcv43hkz8/300w_cropped.zip

--2018-09-02 06:29:50--  https://www.dropbox.com/s/9j2om0gcv43hkz8/300w_cropped.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601a:1::a27d:701
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9j2om0gcv43hkz8/300w_cropped.zip [following]
--2018-09-02 06:29:50--  https://www.dropbox.com/s/raw/9j2om0gcv43hkz8/300w_cropped.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3fdefa3d8ca812b3adf2851185.dl.dropboxusercontent.com/cd/0/inline/APejq8FdpqxF8SZj49FPleXmB703ZUH8F-9anBjwoDzDiAetpbGGge8bcJs7Qd7kmOWfpMCr3AL28ohMpbwmQIGQG4qblmPdfg-EYfIUOEwtPxar8cVX45oQVK6zt5Tpu-JL7EW4wza9PjgZ4SJ3gX-AJpHy9BIkyRB2AkpYjJkGEmlSFKv3u6kqOE7JnCHTiXk/file [following]
--2018-09-02 06:29:51--  https://uc3fdefa3d8ca812b3adf2851185.dl.dropboxusercontent.com/cd/0/inline/APejq8FdpqxF8SZj49FPleXmB703ZUH8F-9anBj

In [0]:
!ls
!unzip 300w_cropped.zip.1
!ls

In [15]:
!ls

01_Indoor   300w_cropped.zip	300w_cropped.zip?dl=0
02_Outdoor  300w_cropped.zip.1	sample_data


In [35]:
from google.colab import files
uploaded = files.upload()

Saving kernel45.txt to kernel45.txt


In [0]:
!ls
!mkdir 300w_samples
!mv indoor* 300w_samples
!ls
!ls 300w_samples

In [20]:
!git clone https://github.com/albanie/pts_loader
!ls pts_loader/
!cp pts_loader/pts_loader.py .
!ls

Cloning into 'pts_loader'...
remote: Counting objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (16/16), done.
LICENSE.md  pts_loader.py  README.md  tests
01_Indoor   300w_cropped.zip	300w_cropped.zip?dl=0  pts_loader.py
02_Outdoor  300w_cropped.zip.1	pts_loader	       sample_data


In [0]:
!ls 01_Indoor/

In [1]:
import os
from os import listdir,makedirs
from os.path import isfile,join
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import tensorflow as tf
import keras.backend as K
import keras
from keras.models import Sequential
import cv2

from keras.layers import Dense, Dropout, Activation, Add, SeparableConv2D
from keras.layers import Convolution2D, Lambda, Input, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Model



Using TensorFlow backend.


In [0]:
'''
Hyper-Parameters
'''
image_H = 96.0
image_W = 96.0
grayscale = True
batch_size = 16
learning_rate=0.001
momentum = 0
decay = 0
epochs = 100
droupout= 0.1
gradient_clipping = 0.1
nFilters = 128
nlandmarks = nLabels = 68
K=45
img_rows = 96
img_cols = 96
img_channels = 1

In [0]:
# Get files
import pts_loader

img_x = []
pts_y = []
for i in range(1,300):
    #print("iteration", i)
    if i == 78 or i == 79 or i == 266:
      continue
    if i < 10:
        img_f = r"01_Indoor/indoor_00" + str(i) +".png"
        pts_f = r"01_Indoor/indoor_00" + str(i) +".pts"
    elif i < 100:
        img_f = r"01_Indoor/indoor_0" + str(i) +".png"
        pts_f = r"01_Indoor/indoor_0" + str(i) +".pts"
    else:
        img_f = r"01_Indoor/indoor_" + str(i) +".png"
        pts_f = r"01_Indoor/indoor_" + str(i) +".pts"
    pts = pts_loader.load(pts_f)
    # Loading imgae in grayscale
    img = cv2.imread(img_f)
    
    scale_x = image_W/img.shape[0]
    scale_y = image_H/img.shape[1]
    img = cv2.resize(img, (img_rows, img_cols))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.normalize(img,None,0,255,cv2.NORM_MINMAX)
    img = img.reshape(img_rows, img_cols, 1)
    landmarks = []
    
    for p in pts:
        new_pts = []
        landmark = np.zeros((96, 96), np.uint8)
        new_pts = (p[0] * scale_y , p[1] * scale_x )
        #new_points.append(new_pts)
        x = int(new_pts[0])
        y = int(new_pts[1])
        #print(n, x, y)
        landmark[x][y] = 1 # Normalized WHite
        landmarks.append(landmark)
        
    img_x.append(img)
    pts_y.append(landmarks)


In [0]:
#Gn = apply_kernel(pts_y)

In [4]:
print(img_x[0].shape)
assert not np.any(np.isnan(img_x))
assert not np.any(np.isnan(pts_y))
train_x = []
train_y = []
test_x = []
test_y = []


(96, 96, 1)


In [0]:
train_x = np.array(img_x[:200])
train_y = np.array(pts_y[:200])
test_x = np.array(img_x[200:])
test_y = np.array(pts_y[200:])


In [6]:
print(train_y.shape)
train_y = train_y.reshape(-1, 96,96,68)
test_y = test_y.reshape(-1, 96,96,68)

print(train_y.shape)

(200, 68, 96, 96)
(200, 96, 96, 68)


In [0]:
def apply_kernel(local_out):
    '''
     get kernel from text file
     tile it to 1x1x45x45
     convolve with local out
    '''
    f = open(r'kernel45.txt')
    kernel45 = np.fromfile(f, dtype = np.float32, count = -1, sep = " ")
    
    # [1 x 1 x 45 x 45]
    kernel45 = np.reshape(kernel45, (45, 45, 1, 1))
    # [numLabels x 1 x 45 x 45] if tensor convolution is used
    kernel_stacked 	= np.tile(kernel45, (1, 1, nLabels, 1))
    kernel_tensor = tf.convert_to_tensor(kernel_stacked, dtype=tf.float32)
    print("local_out", local_out)
    print("kernel", kernel_tensor)

    groups = []
    
    for j in range(nLabels):
        groups.append(tf.nn.conv2d(local_out, kernel_tensor, strides=[1,1,1,1],
                                   padding="SAME"))
    
    heatmaps = tf.concat(groups, 0)
    heatmaps = tf.reshape(heatmaps, (-1, 96, 96, 68))
    print("Heatmaps", heatmaps)
    return heatmaps



In [8]:

'''
LOCAL SUBNET
'''

# [ImageC x ImageH x ImageW]

input_img = Input(( img_rows, img_cols, img_channels))
model = SeparableConv2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(input_img)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(5, 5), padding='same', activation='relu')(model)
model = BatchNormalization()(model)

model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization()(model)
print("Local subnet", model)
'''
1x1 Convolution with activation
'''
model_Conv = Convolution2D(filters=nLabels, kernel_size=(1, 1),activation='relu')(model)
model_Conv = Lambda(apply_kernel, output_shape=(img_cols, img_rows, nLabels))(model_Conv)
print("Main 1x1", model_Conv)
'''
Ol 1x1 Linear convolution
Apply Gaussian Kernel filter
'''
model_Ol = Convolution2D(filters=nLabels, kernel_size=(1, 1),activation=None)(model)
model_Ol = Lambda(apply_kernel, output_shape=(img_cols, img_rows, nLabels), name='Ol')(model_Ol)
print("Local output", model_Ol)

output_Ol = keras.layers.concatenate([model_Conv, model_Ol])
print("Concatenate", output_Ol)
output_Ol = Reshape((96, 96, -1))(output_Ol)
#print("Reshape", output_Ol)

'''
GLOBAL SUBNET
'''
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(output_Ol)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, activation='relu')(model)
model = BatchNormalization()(model)

model = SeparableConv2D(filters=nLabels, kernel_size=(1, 1), activation=None)(model)

FLD_model = Model(inputs = input_img, outputs=model)
FLD_model.summary()


Local subnet Tensor("batch_normalization_15/cond/Merge:0", shape=(?, 96, 96, 128), dtype=float32)
local_out Tensor("conv2d_1/Relu:0", shape=(?, 96, 96, 68), dtype=float32)
kernel Tensor("lambda_1/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Heatmaps Tensor("lambda_1/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
Main 1x1 Tensor("lambda_1/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
local_out Tensor("conv2d_2/BiasAdd:0", shape=(?, 96, 96, 68), dtype=float32)
kernel Tensor("Ol/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Heatmaps Tensor("Ol/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
Local output Tensor("Ol/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
Concatenate Tensor("concatenate_1/concat:0", shape=(?, 96, 96, 136), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (No

* Loss Function: Weighted Square Error
* Exclude non-labeled landmarks from the loss
* Weight each of N landmarks depending on whether they lie within the image boundary:
\begin{equation*}
wij = V (i,j) + \frac{L(i,j)}{10} + \frac{L(i,j)}{2N}\sum_{k=1}^N V (i,k),       \\[1pt]
\text{    where V (i; j) and L(i; j) are indicator functions with L(i; j) = 1 iff landmark j in face i is labeled}    \\[1pt]
\text {V (i; j) = 1 iff landmark j in face i is within the image boundary, zero otherwise.    }\\[1pt]
\text{Note that L(i; j) = 0 implies V (i; j) = 0.}\end{equation*}

\begin{equation*}
e_i = \sum_{n=1}^N w_{in}(((O^l_n - G_n) * K)^2 + (O^g_n - G_n * K)^2), \\[2pt]
\text{    where    } \\[2pt]
O^l_n    and    O^g_n = \text{nth channels of the local- and global context subnet outputs, respectively.} \\[1pt]
\text{K = kernel }   \\[1pt]
\text{Gn = ground truth heatmap for landmark n }   \\[1pt]
\text{For efficiency, the kernel convolution after stacking is reused for the loss}
\end{equation*}


In [0]:
import tensorflow as tf
from keras import backend as K

def landmark_weight(labels):
    # INCOMPLETE
    #[numLabels x 1 x 1]
    Win = tf.reduce_mean(labels)
    Win = tf.multiply(Win, 0.5)
    return Win

'''
Ol - [numLabels x ImageH x ImageW] Kernel conv already applied in Network
Og - [numLabels x ImageH x ImageW]
labels - Landmark coordinates [numLabels x ImageH x ImageW] without
kernel applied
'''
def loss_function(labels, Og):
    
    #Gn = labels #testing
    Gn = apply_kernel(labels)

    #Win = landmark_weight(labels)
    print("---------------Loss Function----------------")
    print("Og", Og)
    print("Labels", labels)
    print("Ground Truth", Gn)
    print("Ol", model_Ol)
    # (Og - Gn)2
    global_diff = tf.subtract(Og, Gn)
    global_err = tf.square(global_diff)
    
    # (Ol - Gn)2
    
    #How to access intermediate layer or model from here???
    #Currently using global variable model_Ol
    
    # Kernel is already applied for model_Ol
    local_diff = tf.subtract(model_Ol, Gn)
    local_err = tf.square(local_diff)
    
    comb_err = tf.add(local_err, global_err)
    
    wt_square_err = tf.reduce_sum(comb_err)
    print("Loss", wt_square_err)
    return wt_square_err    
    #return tf.reduce_sum(global_err) #testing


In [10]:
FLD_model.compile(optimizer='sgd',
              loss=loss_function,
              metrics=['accuracy'])


local_out Tensor("separable_conv2d_23_target:0", shape=(?, ?, ?, ?), dtype=float32)
kernel Tensor("loss/separable_conv2d_23_loss/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Heatmaps Tensor("loss/separable_conv2d_23_loss/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
---------------Loss Function----------------
Og Tensor("separable_conv2d_23/BiasAdd:0", shape=(?, 96, 96, 68), dtype=float32)
Labels Tensor("separable_conv2d_23_target:0", shape=(?, ?, ?, ?), dtype=float32)
Ground Truth Tensor("loss/separable_conv2d_23_loss/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
Ol Tensor("Ol/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
Loss Tensor("loss/separable_conv2d_23_loss/Sum:0", shape=(), dtype=float32)


In [11]:
 FLD_model.fit(train_x, train_y, batch_size=16, epochs=10, validation_data=(test_x, test_y))

Train on 200 samples, validate on 96 samples
Epoch 1/10
200/200 [==============================] - 67s 337ms/step - loss: nan - acc: 0.8345 - val_loss: nan - val_acc: 0.9927
Epoch 2/10
200/200 [==============================] - 55s 275ms/step - loss: nan - acc: 0.9927 - val_loss: nan - val_acc: 0.9927
Epoch 3/10
200/200 [==============================] - 55s 274ms/step - loss: nan - acc: 0.9927 - val_loss: nan - val_acc: 0.9927
Epoch 4/10
200/200 [==============================] - 55s 275ms/step - loss: nan - acc: 0.9927 - val_loss: nan - val_acc: 0.9927
Epoch 5/10
200/200 [==============================] - 55s 274ms/step - loss: nan - acc: 0.9927 - val_loss: nan - val_acc: 0.9927
Epoch 6/10
200/200 [==============================] - 55s 274ms/step - loss: nan - acc: 0.9927 - val_loss: nan - val_acc: 0.9927
Epoch 7/10
200/200 [==============================] - 55s 274ms/step - loss: nan - acc: 0.9927 - val_loss: nan - val_acc: 0.9927
Epoch 8/10
200/200 [==============================] 

In [0]:
import cntk as C

cntk_model = C.load_model(r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\Merget\Merget\python\output\models\model.21")

In [0]:
cntk_model.parameters

In [0]:
cntk_model.save("cntk_model.h5")

In [0]:
FLD = keras.models.load_model(r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\cntk_model.h5")

In [0]:
import keras.backend as K

#save_model_hdf5(cntk_model, 'my_model.h5')
kerasmodel = K.functional_ops. load_model(cntk_model)

In [0]:
#func = cntk_model.outputs[0]
cntk_model.save('cntk_model.onnx', format=C.ModelFormat.)

In [0]:
import onnx
onnx_model = onnx.load('cntk_model.onnx')

In [0]:
from onnx_tf.backend import prepare
tf_model = prepare(onnx_model)